### Trip Adivisor Hotel Reviews 

This [Dataset](https://www.kaggle.com/andrewmvd/trip-advisor-hotel-reviews) from Kaggle(The house of data scientist) 
Usually, The reviewers leave only the messages on the blogpost or any social networks. 
It difficult to interpret or segmentation they are good or bad reviews. Another question, Which topic did the reiviewer talk about. 

This Notebook is cover topic modeling and text segmentation



In [1]:
import numpy as np
import pandas as pd
import tensorflow
# from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
import re
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\natdanai.intraraksa\Anaconda3\lib\site-packages\sklearn\linear_model\_least_angle.py:34: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
C:\Users\natdanai.intraraksa\Anaconda3\lib\site-packages\sklearn\linear_model\_least_angle.py:164: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecation

In [2]:
data = pd.read_csv('../data/tripadvisor/tripadvisor_hotel_reviews.csv')

In [3]:
# Explore dataset to fill NA columns
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20491 entries, 0 to 20490
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  20491 non-null  object
 1   Rating  20491 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 320.3+ KB


In [4]:
data.head()

,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4* experience hotel monaco seat...,3
3,"unique, great stay, wonderful time hotel monac...",5
4,"great stay great stay, went seahawk game aweso...",5


In [5]:
## Extract Reviews sentences from Labels
X = data['Review']
y = data['Rating']
# X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3)

### Topic Modeling

Usually, In the hotel industry. The Enterprenue recived the useful information form data in text uniform such as The comment in social network or email from the customers. 
The Enterprenue may use the useful information and group the useful information for analysis or make the the better goods and 
service

In this section. topic modeling usually use for anaylyze what the topic of the review is?

In [6]:
## Preprocess Text to remove Stopwords

from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
def text_process(text):
    text = re.sub('[^A-za-z]'," ",text)
    text = text.lower()
    text = text.split()
    text = [word for word in text if word not in stopwords.words('english')]
#     text = [SnowballStemmer('english').stem(word) for word in text]
    text = " ".join(text)
    return text

C:\Users\natdanai.intraraksa\Anaconda3\lib\site-packages\sklearn\feature_extraction\image.py:172: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
C:\Users\natdanai.intraraksa\Anaconda3\lib\site-packages\scipy\io\matlab\mio5.py:95: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: http

In [7]:
X_process = X.apply(text_process)

### Topic Modeling by Gensim

In [8]:
import gensim.corpora as corpora
from gensim.models import LdaMulticore

# dictionary = corpora.Dictionary([X_process[:10]])
# dictionary = [corpora.Dictionary([doc.split()]) for doc in X_process[:10]]

C:\Users\natdanai.intraraksa\Anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()
C:\Users\natdanai.intraraksa\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [9]:
temp = []
for doc in X_process:
    temp.append(doc.split())

In [10]:
dictionary = corpora.Dictionary(temp)

In [11]:
corpus = [dictionary.doc2bow(doc.split()) for doc in X_process[:10]]

In [12]:
# corpora.Dictionary([X_process[0]])
lda = LdaMulticore(corpus=corpus,num_topics=5,id2word=dictionary)

In [13]:
lda.print_topics()

[(0,
  '0.001*"room" + 0.001*"hotel" + 0.000*"nice" + 0.000*"great" + 0.000*"stay" + 0.000*"night" + 0.000*"good" + 0.000*"suite" + 0.000*"n" + 0.000*"bed"'),
 (1,
  '0.001*"room" + 0.001*"hotel" + 0.001*"n" + 0.001*"staff" + 0.001*"great" + 0.000*"stay" + 0.000*"suite" + 0.000*"desk" + 0.000*"nice" + 0.000*"good"'),
 (2,
  '0.001*"hotel" + 0.000*"room" + 0.000*"staff" + 0.000*"stay" + 0.000*"great" + 0.000*"suite" + 0.000*"monaco" + 0.000*"n" + 0.000*"nice" + 0.000*"reception"'),
 (3,
  '0.000*"hotel" + 0.000*"room" + 0.000*"great" + 0.000*"n" + 0.000*"staff" + 0.000*"stay" + 0.000*"service" + 0.000*"like" + 0.000*"suite" + 0.000*"good"'),
 (4,
  '0.000*"room" + 0.000*"hotel" + 0.000*"great" + 0.000*"n" + 0.000*"stay" + 0.000*"good" + 0.000*"building" + 0.000*"desk" + 0.000*"suite" + 0.000*"bath"')]

### Multiclass Classification

In [14]:
from tensorflow.keras.preprocessing.text import Tokenizer,one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense
from tensorflow.keras.utils import to_categorical

In [17]:
from transformers import BertTokenizer,BertForSequenceClassification,TFBertModel,glue_convert_examples_to_features

In [18]:
bert_token = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

ImportError: 
BertForSequenceClassification requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.


In [ ]:
token = Tokenizer()

In [ ]:
token.fit_on_texts(X_train)

In [ ]:
x_train = token.texts_to_sequences(X_train)
x_test = token.texts_to_sequences(X_test)

In [ ]:
x_train = pad_sequences(x_train,maxlen=150,padding='post')
x_test = pad_sequences(x_test,maxlen=150,padding='post')

In [ ]:
y_train = to_categorical(y_train,num_classes=6)
y_test = to_categorical(y_test,num_classes=6)

In [ ]:
vocab_size = len(token.word_index) + 1 

model = Sequential()
model.add(Embedding(input_dim = vocab_size,output_dim = 64,input_length = 150))
model.add(LSTM(128,dropout = 0.2))
model.add(Dense(6,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(x_train,y_train,batch_size=128,epochs=113,validation_data=(x_test,y_test))

In [ ]:
data['Rating'][0]

In [ ]:
14343/128

In [ ]:
a[15]

In [ ]:
pd.value_counts(data['Rating'])

In [ ]:
data['Rating'].min()